<a href="https://colab.research.google.com/github/carolineoliveira994/100-days-of-code-python/blob/main/deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import json
from tqdm import tqdm

In [ ]:
class Tokenizer:

  def _init_(self):
    self.dicionario = {}
    self.dicionario_inverso = {}
    self.token_desconhecido = 1
    self.token_padding = 0
    self.max_len = 0

  def fit(self, dataset):
    indice = 2

    for texto in dataset:
      palavras = texto.split(' ')
      self.max_len = len(palavras) if len(palavras) > self.max_len else self.max_len
      for palavra in palavras:
        if palavra not in self.dicionario:
          self.dicionario[palavra] = indice
          indice += 1

    self.dicionario_inverso = {
        valor : chave for chave, valor in dicionario.items()

    }

  def encode(self, texto):
    tokens = texto.split('')
    texto_tokenizado = []
    for token in tokens:
      if token not in self.dicionario.keys():
        texto_tokenizado.append(self.token_desconhecido)
      else:
        texto_tokenizado.append(self.dicionario[token])
    #from keras.preprocessing.sequence import pad_sequences
    #texto_tokenizado = pad_sequences([texto_tokenizado], maxlen=self.max_len, padding='post')
    for _ in range(self.max_len - len(texto_tokenizado)):
      texto_tokenizado.append(self.token_padding)
    return texto_tokenizado



  def decode(self, tokens):
    texto = []
    for token in tokens:
      if token == self.token_padding:
        texto.append('PAD')
      elif token not in self.dicionario_inverso:
        texto.append(self.token_desconhecido)
      else:
        texto.append(self.dicionario_inverso[token])

    return ''.join(texto)

  def save(self):
    with open('dicionario.json', 'w') as f:
      json.dump(self.dicionario, f)
    with open('dicionario_inverso,json', 'w') as f:
      json.dump(self.dicionario_inverso, f)

    #salvando o max_len
    with open('max_len.json', 'w') as f:
      json.dump(self.max_len, f)


  def load(self):
    with open('dicionario.json', 'r') as f:
      self.dicionario = json.load(f)
    with open ('dicionario_inverson.json', 'r') as f:
      self.dicionario_inverso = json.load(f)








In [ ]:
dataset = [
  "bom celular",
  "sol, choveu durante o dia",
  "bom dia, não gostei disso"
]

tokenizer = Tokenizer()
tokenizer.fit(dataset)
tokenized_dataset = [tokenizer.encode(x) for x in dataset]
tokenized_dataset

AttributeError: 'Tokenizer' object has no attribute 'max_len'

quais camadas a rede tem?

In [ ]:
class RNN(nn.Module):
  def _init_(self, input_size, hidden_size, output_size):
    super(RNN, self)._init_()
    self.input = nn.Linear(input_size, hidden_size)
    self.hidden = nn.Linear(hidden_size, hidden_size)
    self.output = nn.Linear(hidden_size, output_size)

    self.hidden_size = hidden_size

  def forward(self, input, hidden):
    input = self.input(input)
    if hidden is None:
      hidden = self.initHidden()
    hidden = self.hidden(hidden)
    hidden = nn.Tanh()(hidden+input)
    output = self.output(hidden)
    return output, hidden


  def initHidden(self):
    return torch.zeros(1, self.hidden_size)





LSTM



In [ ]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(LSTM, self).__init__()
    self.input = nn.Linear(input_size, hidden_size)
    self.output = nn.Linear(hidden_size, output_size)

    self.ft = nn.Linear(hidden_size * 2, hidden_size)
    self.it = nn.Linear(hidden_size * 2, hidden_size)
    self.ct = nn.Linear(hidden_size * 2, hidden_size)
    self.ot = nn.Linear(hidden_size * 2, hidden_size)

    self.sigmoid = nn.Sigmoid()  # Corrected typo (should be 'sigmoid')
    self.tanh = nn.Tanh()
    self.hidden_size = hidden_size

  def forward(self, input, state=None):
    if state is None:
      hidden, cell = self.initHidden()
    else:
      hidden, cell = state

    input = self.input(input)

    ft = torch.cat((hidden, input), 1)
    ft = self.sigmoid(self.ft(ft))

    it = torch.cat((hidden, input), 1)
    it = self.sigmoid(self.it(it))

    ct = torch.cat((hidden, input), 1)
    ct = self.tanh(self.ct(ct))

    ct = ft * cell + it * ct

    ot = torch.cat((hidden, input), 1)
    ot = self.sigmoid(self.ot(ot))

    ht = ot * self.tanh(ct)
    output = self.output(ht)

    return output, [ht, ct]

  def initHidden(self):
    hidden_state = torch.zeros(1, self.hidden_size)
    cell_state = torch.zeros(1, self.hidden_size)  # Corrected typo (torch.zeros)
    return hidden_state, cell_state


In [ ]:
class GRU(nn.Module):
  def _init_(self, input_size, hidden_size):
    super(GRU, self)._init_()
    self.input = nn.Linear(input_size, hidden_size)
    self,output = nn.Linaer(hidden_size, output_size)

    self.zt = nn.Linear(hidden_size * 2, hidden_size)
    self.rt = nn.Linear(hidden_size * 2, hidden_size)
    self.ht = nn.Linear(hidden_size * 2, hidden_size)
    self.tanh = nn.Tanh()

    self.sigmoide = nn.Sigmoide()

  def forward(self, input, hidden):
    if hidden is None:
      hidden = self.initHid

    input = self.input(input)

    zt = torch.cat((hidden, input), 1)
    zt = self.sigmoide(self.zt(zt))

    rt = torch.cat((hidden, input), 1)
    rt = self.sigmoide(self.rt(rt))

    ht = torch.cat((hidden, input), 1)
    ht = self.tanh(self.ht(ht))

    ht = (1 - zt) * hidden * zt * ht
    return output, ht

  def initHidden(self):
    return torch.zeros(1, self.hidden_size)

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        num_tokens,
        dim_model,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dropout_p,
    ):
        super().__init__()

        self.positional_encoder = PositionalEncoding(
            dim_model=dim_model, dropout_p=dropout_p, max_len=5000
        )
        self.embedding = nn.Embedding(num_tokens, dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
        )
        self.output = nn.Linear(dim_model, num_tokens)  # Changed for correct output

    def forward(self, src, tgt, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None):
        src = self.embedding(src) * math.sqrt(self.dim_model)
        tgt = self.embedding(tgt) * math.sqrt(self.dim_model)
        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)

        src = src.permute(1, 0, 2)  # Permute for sequence-first format
        tgt = tgt.permute(1, 0, 2)

        output_transformer = self.transformer(
            src,
            tgt,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_pad_mask,
            tgt_key_padding_mask=tgt_pad_mask,
        )
        output = self.output(output_transformer)  # Apply output layer
        return output

    def get_tgt_mask(self, size) -> torch.tensor:
        mask = torch.tril(torch.ones(size, size))
        mask = mask.masked_fill(mask == 0, float("-inf"))  # Corrected typo "mak" to "mask"
        mask = mask.masked_fill(mask == 1, float(0.0))  # Changed float(0,0) to float(0.0)
        return mask


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Transformer(
    num_tokens=len(tokenizer.dicionario.keys()) + 4,
    dim_model=32,
    num_heads=2,
    num_encoder_layers=3,
    num_decoder_layers=3,
    dropout_p=0.1
).to(device)
opt = torch.optim.SGD(model.parametrs(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
model

AttributeError: 'Tokenizer' object has no attribute 'dicionario'

In [ ]:
model.train()
for _ in range(1):
  for sentence in tokenized_dataset:
    x, y = sentence, sentence
    x, y = torch.tensor(x).to(device)[None], torch.tensor(y).to(device)[None]

    y_input = y[:, :-1]
    y_expected = y[:, 1:]

    tgt_mask = model.get_tgt_mask(y_input.size(1)).to(device)

    pred = model(x, y, tgt_mask)

    pred = prd.permute(1, 2, 0)
    loss_fr(pred, y_expected)

    opt.zero_grad()
    loss.backward()
    opt.step()



NameError: name 'model' is not defined

In [ ]:
torch.tril(torch.ones(5,5))
mask

In [ ]:
import pytorch_lightning as pl
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import dataloader
import torch.nn as nn

In [ ]:
%%capture
!pip install pytorch-lightning

In [ ]:
import pytorch_lightning as pl
import torchmetrics
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

In [ ]:
class Net(pl.LightningModule):
  def _init_(self):
    super()._init_()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16 * 5 *5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

    self.train_acc = pl.metrics.Accuracy()
    self.criterion = nn.CrossEntropyLoss()

  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

  def training_step(self, batch, batch_idx):
    inputs, labels = batch
    pred = self.forward(inputs)

    loss = self.criterion(pred, labels)
    return loss

  def forward(self, x):
    x = self.conv1(x)
    x = self.pool(F.relu(x))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flattenn(x, 1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x







In [ ]:
class CIFARDataMModule(pl.LightningDataModule):
  def _init_(self, batch_size=128, transform=None):
    super()._init_()
    self.batch_size = batch_size
    self.transform = transform

  def prepare_data(self):
    CIFAR10(root='data_train', train=True, download=True)
    CIFAR10(root='data_train', train=False, download=True)

  def setup(self, stage):
    training_dataset = CIFAR10('data_train', download=False, train=True, transform = self.transform)
    test_dataset = CIFAR10('data_test', download=False, train=False, transform = self.transform)
    train, eval = random_split(training_dataset, [
        int(len(training_dataset) * 0.7),
        int(len(training_dataset) * 0.3)

    ])

    self.train_dataset = train
    self.eval_dataset = eval
    self.test_dataset = test_dataset

  def train_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=True)

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=True)


In [ ]:
model = Net()
transform = transforms.ToTensor()
cifar_dm = CIFARDataModule(batch_size=128, transform=transform)

trainer = pl.Trainer(max_epochs=2, gpus=0)
trainer.fit(model, cifar_dm)